# Generate Test Subsets

In [1]:
import itertools
import random
from itertools import combinations
from random import sample

import numpy as np
import pandas as pd
import tqdm

In [2]:
string_df = pd.read_csv("../../data/interim/string_df.csv")

In [3]:
string_df

,label,concatenated
0,GooglePixel3A_L,0000000000000000000000000000000000000000000000...
1,GooglePixel3A_L,0000000000000000000000000000000000000000000000...
2,GooglePixel3A_L,0000000000000000000000000000000000000000000000...
3,GooglePixel3A_L,0000000000000000000000000000000000000000000000...
4,GooglePixel3A_L,0000000000000000000000000000000000000000000000...
...,...,...
951,iPhoneXSMax_M,0001101000101101000000000001101111111111000000...
952,iPhoneXSMax_M,0001101000101101010000000001101111111111000000...
953,iPhoneXSMax_M,0001101000101101000000000001101111111111000000...
954,iPhoneXSMax_M,0001101000101101010000000001101111111111000000...


In [4]:
# Identifying unique labels
unique_labels = string_df['label'].unique()

# Initializing lists to store matching and non-matching pairs
matching_pairs = []
non_matching_pairs = []

# Generating matching pairs
for label in unique_labels:
    # Getting all indices for the current label
    indices = string_df[string_df['label'] == label].index.tolist()
    if len(indices) == 1:
        # If only one row, duplicate it to create 50 pairs
        indices = indices * 2
    matching_indices = list(itertools.combinations(indices, 2))
    
    # Ensuring we have exactly 50 pairs
    if len(matching_indices) < 50:
        matching_indices = matching_indices * (50 // len(matching_indices) + 1)
    matching_pairs.extend([(i, j, label, label) for i, j in matching_indices[:50]])

# Generating non-matching pairs
all_indices = list(string_df.index)
for label in unique_labels:
    # Indices with the current label
    label_indices = string_df[string_df['label'] == label].index.tolist()
    # Indices without the current label
    other_indices = list(set(all_indices) - set(label_indices))
    
    # Creating pairs between label indices and other indices
    non_matching_indices = list(itertools.product(label_indices, other_indices))
    
    # Ensuring we have exactly 50 pairs
    if len(non_matching_indices) < 50:
        non_matching_indices = non_matching_indices * (50 // len(non_matching_indices) + 1)
    non_matching_pairs.extend([(i, j, label, string_df.loc[j, 'label']) for i, j in non_matching_indices[:50]])

# Creating the final dataframe
pairs_df = pd.DataFrame(matching_pairs + non_matching_pairs, columns=['index_1', 'index_2', 'label_1', 'label_2'])
print(pairs_df)

NameError: name 'itertools' is not defined

In [ ]:
pairs_df

,index1,index2,label1,label2
0,18,38,GooglePixel3A_L,GooglePixel3A_L
1,30,14,GooglePixel3A_L,GooglePixel3A_L
2,6,39,GooglePixel3A_L,GooglePixel3A_L
3,29,26,GooglePixel3A_L,GooglePixel3A_L
4,1,17,GooglePixel3A_L,GooglePixel3A_L
...,...,...,...,...
1245,929,621,iPhoneXSMax_M,XiaomiRedmiNote9S_T
1246,945,187,iPhoneXSMax_M,SamsungJ6_K
1247,946,129,iPhoneXSMax_M,OnePlusNord_O
1248,951,463,iPhoneXSMax_M,XiaomiRedmi5_J


In [ ]:
string_df['label'].nunique()

33